In [ ]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt

import typing

## Functions and data

In [ ]:

def plot_multiple_histories(histories : typing.Dict, metric='loss'):
  """
  Plots the training history of multiple models on the same plot for comparison.
  
  Parameters:
    histories: List of History objects from multiple model's fit() calls.
    metric: The metric to plot ('loss', 'accuracy', etc.).
    labels: List of labels for the models. If None, defaults to 'Model 1', 'Model 2', etc.
  """
  # Check if labels are provided, otherwise generate default labels
  # if labels is None:
  #   labels = [f'Model {i+1}' for i in range(len(histories))]
  
  # Initialize the plot
  plt.figure(figsize=(10, 6))
  
  # Loop through each history and plot the metrics
  for i, (label, history) in enumerate(histories.items()):
    epochs = range(1, len(history.history[metric]) + 1)
    
    # Plot training metric
    plt.plot(epochs, history.history[metric], label=f'{label} {metric.capitalize()}')
    
    # Plot validation metric if available
    if f'val_{metric}' in history.history:
      plt.plot(epochs, history.history[f'val_{metric}'], '--', label=f'{label} Validation {metric.capitalize()}')
  
  # Add labels and title
  plt.title(f'Comparison of {metric.capitalize()} Between Models')
  plt.xlabel('Epochs')
  plt.ylabel(metric.capitalize())
  plt.legend()
  
  # Show plot
  plt.show()



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

X_r,y_r = make_regression(
  n_samples=5000,
  n_features=4,
  n_informative=3,
  noise=2
)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
  X_r,
  y_r,
  test_size=0.33,
  random_state=42
)

from sklearn.datasets import make_classification

X_c,y_c = make_classification(n_samples=1000, n_features=4, n_redundant=0)

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_r, y_r, test_size=0.33, random_state=42)

## Learning Rate

In [ ]:
def scheduler(epoch, lr, cutoff_epoch) -> float:
  if epoch < cutoff_epoch:
    return float(lr)
  else:
    return float(lr * tf.math.exp(-0.1))
  
callback = keras.callbacks.LearningRateScheduler(scheduler)

def train_model(callback) -> keras.callbacks.History:
  model = keras.models.Sequential()
  model.add(keras.layers.InputLayer(shape=(4,)))
  model.add(keras.layers.Dense(1))
  opt = keras.optimizers.SGD(learning_rate=0.001)
  model.compile(optimizer=opt, loss='mse', metrics=['mae'])
  history = model.fit(
    X_train_r, 
    y_train_r,
    batch_size=128, 
    epochs=100, 
    validation_split=0.3,
    callbacks=[callback]
  )
  return history



In [ ]:
plot_multiple_histories(
  histories={
    f"epoch{i:3d}" : train_model(
      keras.callbacks.LearningRateScheduler(lambda e, lr : scheduler(e, lr, i))
    )
    for i in range(0, 100, 10)
  }
)

## Early Stopping

In [ ]:

# This callback will stop the training when there is no improvement 
# in the validation loss for three consecutive epochs.
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model = keras.Sequential([keras.layers.Dense(10)])
model.compile(keras.optimizers.SGD(), loss='mse')
history = model.fit(
  np.arange(100).reshape(5, 20), 
  np.zeros(5),
  epochs=10, 
  batch_size=1, 
  callbacks=[callback]
)

len(history.history['loss'])  # Only 4 epochs are run.


## Weight Regularization

In [ ]:

from sklearn.datasets import make_classification

X,y = make_classification(n_samples=1000, n_features=4, n_redundant=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [ ]:


def train_model_regular(regularizer_rate=1.0) -> keras.callbacks.History:
  model = keras.models.Sequential()
  model.add(keras.Input(shape=(4,)))
  model.add(keras.layers.Dense(10))
  model.add(
    keras.layers.Dense(
      10,
      kernel_regularizer=keras.regularizers.l2(regularizer_rate)
    )
  )
  model.add(keras.layers.Dense(1, activation='sigmoid'))
  
  model.compile(
    optimizer='SGD',
    loss='binary_crossentropy',
    metrics=['accuracy']
  )
  history = model.fit(
    X_train, 
    y_train,
    batch_size=32, 
    epochs=40, 
    validation_split=0.3
  )
  
  return history



In [ ]:
plot_multiple_histories(
  histories={
    "0.001" : train_model_regular(0.001),
    "0.01" : train_model_regular(0.01),
    "0.1" : train_model_regular(0.1),
  }
)

## Dropout


In [ ]:


def train_model_dropout(dropout_rate) -> keras.callbacks.History:
  model = keras.models.Sequential()
  model.add(keras.Input(shape=(4,)))
  model.add(keras.layers.Dropout(dropout_rate))
  model.add(keras.layers.Dense(10))
  model.add(keras.layers.Dropout(dropout_rate))
  model.add(keras.layers.Dense(1, activation='sigmoid'))
  
  model.compile(
    optimizer='SGD',
    loss='binary_crossentropy',
    metrics=['accuracy']
  )
  history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=40,
    validation_split=0.3
  )
  
  return history


In [ ]:
plot_multiple_histories(
  histories={
    f"{val:f}" : train_model_regular(0.001)
    for val in np.arange(0.1, 1.01, 0.2)
  }
)